In [ ]:
import pickle

import numpy as np
from sklearn.ensemble import (
    RandomForestRegressor,
    RandomForestClassifier,
    StackingRegressor,
)
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression


import pandas as pd
from src.utils import save

verbose = 0

In [ ]:
"""
Inspired by Franck Zibi's model :
Level 0 : Extract features from the series of data
Level 1 : Base classification with GradientBoostingClassifier
Level 2 : Residual estimation of the probabilities with ensemble of models
    - For each category
        - RandomForestRegressor
        - KNeighborsRegressor
        - MLPRegressor
        - SVR
Level 3 : Stacking of the residual estimations
    - For each category
        - StackingRegressor with GradientBoostingRegressor
Level 4 : Nothing for now, but calibration on the test set could be done
"""

# Level 1

base_classifier = RandomForestClassifier(verbose=verbose, n_jobs=18)

# Level 2
regressors = []
for _ in range(24):
    regressors.append(
        [
            (
                "RandomForestRegressor",
                RandomForestRegressor(verbose=verbose, n_jobs=18),
            ),
            ("KNeighborsRegressor", KNeighborsRegressor(n_jobs=18)),
            ("LinearRegression", LinearRegression(n_jobs=18)),
        ]
    )

# Level 3
stacking_regressors = []
for k in range(24):
    stacking_regressors.append(
        StackingRegressor(
            estimators=regressors[k],
            final_estimator=LinearRegression(n_jobs=18),
        )
    )

In [ ]:
X_train = pd.read_parquet(f"data/features_train.parquet")
y_train = pd.read_csv("data/y_train.csv")
X_test = pd.read_parquet(f"data/features_test.parquet")

X_train = X_train.values[:, 1:]
y_train = y_train.values[:, 1]
X_test = X_test.values[:, 1:]

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1)

print(X_train.shape, X_val.shape, y_train.shape, y_val.shape)

In [ ]:
one_hot_y_val = np.eye(np.max(y_val) + 1)[y_val]
one_hot_y_train = np.eye(np.max(y_train) + 1)[y_train]

print("Training base classifier...")
base_classifier.fit(X_train, y_train)

y_train_residuals = one_hot_y_train - base_classifier.predict_proba(X_train)

for k in range(24):
    print(f"Training regressor {k}...")
    stacking_regressors[k].fit(X_train, y_train_residuals[:, k])

In [ ]:
with open("runs/simple/base_classifier.pkl", "wb") as f:
    pickle.dump(base_classifier, f)

for k in range(24):
    with open(f"runs/simple/stacking_regressor_{k}.pkl", "wb") as f:
        pickle.dump(stacking_regressors[k], f)

In [ ]:
with open("runs/simple/base_classifier.pkl", "rb") as f:
    base_classifier = pickle.load(f)

for k in range(24):
    with open(f"runs/simple/stacking_regressor_{k}.pkl", "rb") as f:
        stacking_regressors[k] = pickle.load(f)

In [ ]:
y_val_base_pred = base_classifier.predict_proba(X_val)
y_val_residuals = np.zeros_like(y_val_base_pred)
for k in range(24):
    y_val_residuals[:, k] = stacking_regressors[k].predict(X_val)
y_val_pred = y_val_base_pred + y_val_residuals
np.save("runs/simple/val_predictions.npy", y_val_pred)
y_val_pred = np.argmax(y_val_pred, axis=1)

print("Validation accuracy:", (y_val_pred == y_val).mean())

In [ ]:
y_test_base_pred = base_classifier.predict_proba(X_test)
y_test_residuals = np.zeros_like(y_test_base_pred)
for k in range(24):
    y_test_residuals[:, k] = stacking_regressors[k].predict(X_test)
y_test_pred = y_test_base_pred + y_test_residuals
np.save("runs/simple/test_predictions.npy", y_test_pred)
y_test_pred = np.argmax(y_test_pred, axis=1)

save(y_test_pred)

In [ ]:
y_train_base_pred = base_classifier.predict_proba(X_train)
y_train_residuals = np.zeros_like(y_train_base_pred)
for k in range(24):
    y_train_residuals[:, k] = stacking_regressors[k].predict(X_train)
y_train_pred = y_train_base_pred + y_train_residuals
np.save("runs/simple/train_predictions.npy", y_train_pred)

print("Training accuracy:", (np.argmax(y_train_pred, axis=1) == y_train).mean())